In [209]:
import pandas as pd
import numpy as np
import json

#jsonpath = os.path.join('', 'purchase_data.json')



df = pd.read_json('purchase_data.json')
df_2 = pd.read_json('purchase_data2.json')

df = pd.merge(df,df_2,how='outer')
df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [210]:
player_count = len(df['SN'].unique())
player_count

612

In [211]:
#Purchasing Analysis
unique_items = len(df['Item Name'].unique())
avg_purchase_price = round(df['Price'].mean(),2)
total_purchases = df['Item ID'].count()
total_revenue = round(df['Price'].sum(),2)


df_purchase = pd.DataFrame({'Unique Items':unique_items,
             'Average Purchase Price':avg_purchase_price,
             'Total Purchases':total_purchases,
             'Total Value':[total_revenue]})

df_purchase = df_purchase[['Unique Items','Average Purchase Price','Total Purchases','Total Value']]
df_purchase

,Unique Items,Average Purchase Price,Total Purchases,Total Value
0,180,2.93,858,2514.43


In [212]:
#Gender Demographics

male = df['Gender'] == 'Male'
female = df['Gender'] == 'Female'

df_groupby_gender = df.groupby(['Gender'])

gender_count = df_groupby_gender['SN'].nunique()
gender_percent = round(gender_count/player_count*100,2).map("{0:.2f}%".format)

df_gender_demo = pd.DataFrame({'Purchasers Count':gender_count,
                         'Percent':gender_percent})
df_gender_demo = df_gender_demo[['Purchasers Count','Percent']]
df_gender_demo

,Purchasers Count,Percent
Gender,,
Female,112,18.30%
Male,498,81.37%
Other / Non-Disclosed,9,1.47%


In [213]:
#Purchasing Analysis (gender)

df_groupby_gender = df.groupby(['Gender'])

gender_purchase_count = df['Gender'].value_counts()
gender_avg_purchase_price = df_groupby_gender['Price'].mean().map("$ {:,.2f}".format)
gender_total_value = df_groupby_gender['Price'].sum().map("$ {:,.2f}".format)

df_gender = pd.DataFrame({'Purchase Count':gender_purchase_count,
                         'Average Purchase Price':gender_avg_purchase_price,
                         'Total Value':gender_total_value})
df_gender = df_gender[['Purchase Count','Average Purchase Price','Total Value']]
df_gender

,Purchase Count,Average Purchase Price,Total Value
Female,149,$ 2.85,$ 424.29
Male,697,$ 2.94,"$ 2,052.28"
Other / Non-Disclosed,12,$ 3.15,$ 37.86


In [214]:
#Age demographics

bins = np.arange(df['Age'].min(),df['Age'].max()+df['Age'].min(),4)
label = ['7-11','11-15','15-19','19-23','23-27','27-31','31-35','35-39','39-43','43-47','47-51']

df['Age Groups'] = pd.cut(df['Age'],bins,labels=label)
df_groupby_age = df.groupby(['Age Groups'])
age_purchase_count = df_groupby_age['Age'].count()
age_avg_purchase = df_groupby_age['Price'].mean().map("$ {:,.2f}".format)
age_total_revenue = df_groupby_age['Price'].sum().map("$ {:,.2f}".format)


df_age = pd.DataFrame({'Purchase Count':age_purchase_count,
                      'Average Purchase':age_avg_purchase,
                      'Total Value':age_total_revenue})
df_age = df_age[['Purchase Count','Average Purchase','Total Value']]
df_age

,Purchase Count,Average Purchase,Total Value
Age Groups,,,
7-11,28,$ 3.12,$ 87.28
11-15,71,$ 2.86,$ 202.76
15-19,96,$ 2.85,$ 273.22
19-23,295,$ 2.91,$ 858.33
23-27,183,$ 2.98,$ 544.95
27-31,64,$ 2.94,$ 188.17
31-35,49,$ 3.00,$ 147.11
35-39,34,$ 2.87,$ 97.45
39-43,17,$ 3.28,$ 55.68


In [215]:
# Top Spenders
df_groupby_sn = df.groupby(['SN'])
purchaser_count = df['SN'].value_counts()
purchaser_avg_price = round(df_groupby_sn['Price'].mean(),2)
purchaser_total_revenue = round(df_groupby_sn['Price'].sum(),2)
df_sn = pd.DataFrame({'Purchases': purchaser_count,
                     'Average Purchase Price':purchaser_avg_price,
                     'Total Value':purchaser_total_revenue})
df_sn = df_sn[['Purchases','Average Purchase Price','Total Value']]
df_sn = df_sn.sort_values(['Total Value'],ascending = False)
df_sn.head(5)

,Purchases,Average Purchase Price,Total Value
Undirrala66,5,3.41,17.06
Aerithllora36,4,3.78,15.10
Saedue76,4,3.39,13.56
Sondim43,4,3.26,13.02
Mindimnya67,4,3.18,12.74


In [216]:
#Most Popular items
df_groupby_item = df.groupby(['Item ID','Item Name'])
#item_id = round(df_groupby_item['Item ID'].mean(),0)
item_count = df_groupby_item['Item Name'].count()
item_price = round(df_groupby_item['Price'].mean(),2)
item_purchase_value = round(df_groupby_item['Price'].sum(),2)

df_item = pd.DataFrame({
                        'Purchases':item_count,
                       'Price':item_price,
                       'Total Value':item_purchase_value})
df_item = df_item[['Purchases','Price','Total Value']]
df_item = df_item.sort_values(['Purchases'],ascending=False)
df_item.head(5)

,,Purchases,Price,Total Value
Item ID,Item Name,,,
84,Arcane Gem,12,2.45,29.34
39,"Betrayal, Whisper of Grieving Widows",11,2.35,25.85
31,Trickster,10,2.32,23.22
44,Bonecarvin Battle Axe,9,2.67,24.04
154,Feral Katana,9,2.62,23.55


In [217]:
#Most Profitable items
df_item_profit = df_item.sort_values(['Total Value'],ascending=False)
df_item_profit.head(5)

,,Purchases,Price,Total Value
Item ID,Item Name,,,
34,Retribution Axe,9,4.14,37.26
107,"Splitter, Foe Of Subtlety",9,3.67,33.03
115,Spectral Diamond Doomblade,7,4.25,29.75
32,Orenmir,6,4.95,29.70
84,Arcane Gem,12,2.45,29.34


In [218]:
print('Purchasing Analysis')
print('----------------------------')
print(df_purchase)
print('----------------------------')
print('Gender Demographics')
print('----------------------------')
print(df_gender_demo)
print('----------------------------')
print('Purchasing Analysis by Gender')
print('----------------------------')
print(df_gender)
print('----------------------------')
print('Purchasing Analysis by Age')
print('----------------------------')
print(df_age)
print('----------------------------')
print('Top Spenders')
print('----------------------------')
print(df_sn.head(5))
print('----------------------------')

print('Most Popular Items')
print('----------------------------')
print(df_item.head(5))
print('----------------------------')
print('Most Profitable Items')
print('----------------------------')
print(df_item_profit.head(5))

Purchasing Analysis
----------------------------
   Unique Items  Average Purchase Price  Total Purchases  Total Value
0           180                    2.93              858      2514.43
----------------------------
Gender Demographics
----------------------------
                       Purchasers Count Percent
Gender                                         
Female                              112  18.30%
Male                                498  81.37%
Other / Non-Disclosed                 9   1.47%
----------------------------
Purchasing Analysis by Gender
----------------------------
                       Purchase Count Average Purchase Price Total Value
Female                            149                 $ 2.85    $ 424.29
Male                              697                 $ 2.94  $ 2,052.28
Other / Non-Disclosed              12                 $ 3.15     $ 37.86
----------------------------
Purchasing Analysis by Age
----------------------------
            Purchase Count A